In [1]:
!sudo apt-get install curl autoconf automake libtool pkg-config
!git clone https://github.com/openvenues/libpostal
%cd libpostal
!./bootstrap.sh
!./configure --datadir="/content"
!make -j4
!sudo make install
!sudo ldconfig

!sudo apt-get install python3
!sudo apt-get install python3-pip

!python3 --version
!pip3 --version

!pip3 install postal

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autoconf is already the newest version (2.71-2).
autoconf set to manually installed.
automake is already the newest version (1:1.16.5-1.3).
automake set to manually installed.
curl is already the newest version (7.81.0-1ubuntu1.21).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
Suggested packages:
  libtool-doc gcj-jdk
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libtool pkg-config
0 upgraded, 2 newly installed, 2 to remove and 41 not upgraded.
Need to get 212 kB of archives.
After this operation, 1,223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pkg-config amd64 0.29.2-1ubuntu3 [48.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtool all 2.4.

In [4]:
!pip install -U sentence-transformers scikit-learn pandas pyarrow matplotlib phonenumbers tldextract
!pip install fuzzywuzzy python-levenshtein

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
  Using cached python_levenshtein-0.27.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached levenshtein-0.27.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 48.6 MB/s eta 0:00:00


In [1]:
# ===================================================================
# Step 1: Environment Setup & Library Installation
# ===================================================================
# Install required libraries:
# - sentence-transformers: to load pre-trained sentence embedding models
# - pyarrow: for reading parquet files
# - pandas, scikit-learn, matplotlib: standard data analysis & ML stack

import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import phonenumbers
import tldextract
import json
import re # Used for cleaning the input string before parsing
from sklearn.ensemble import GradientBoostingClassifier

import torch

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint # For integer distributions
from scipy.stats import uniform as sp_uniform # For float distributions


# Scikit-learn imports for modeling and evaluation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_recall_curve, auc, classification_report, roc_auc_score

# SentenceTransformers imports
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from postal.expand import expand_address

# Optional: Check if a GPU is available in Colab
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device in use: {device.upper()}")

Device in use: CUDA


Define Helper Functions

In [2]:
def safe_json_parse(text, key):
    # A placeholder for the function that extracts a value from JSON/complex strings
    if not text: return ""
    try:
        data = json.loads(text)
        if isinstance(data, dict): return str(data.get(key, ""))
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
            return str(data[0].get(key, ""))
        return ""
    except (json.JSONDecodeError, TypeError):
        # When JSON parsing fails, it may return the original text if it's the only value
        # For the purpose of locality extraction, we should NOT return the full text here.
        return ""

def normalize_phone(text, country_code):
    """
    Extract and normalize phone numbers to E.164 format (+CCNNNNNNNNN) using the provided country code.

    Parameters
    ----------
    text: str
        List of phone numbers (usually a single list)

    country code: str
        Country code of the input phone number (ex. "GB")

    Returns
    -------
    set : A normalized phone number in E.164 format format.
    """
    if not text or not country_code:
        return ""
    try:
        data = json.loads(text)
        phone_str = None

        # Extract the raw phone string from the list structure (similar to previous logic)
        if isinstance(data, list) and len(data) > 0:
            first_element = data[0]
            if isinstance(first_element, str):
                phone_str = first_element
            elif isinstance(first_element, dict):
                phone_str = first_element.get('value', first_element.get('phone'))

            if phone_str and isinstance(phone_str, str):
                # 1. Attempt to parse using the known country_code
                parsed_num = phonenumbers.parse(phone_str, country_code)

                # 2. If valid, format it to the unique E.164 string
                if phonenumbers.is_valid_number(parsed_num):
                    return phonenumbers.format_number(parsed_num, phonenumbers.PhoneNumberFormat.E164)

        return ""
    except (json.JSONDecodeError, TypeError, phonenumbers.NumberParseException):
        # Catch any data parsing or number parsing errors
        return ""

# Assuming you've already defined safe_json_parse, add this simple extraction function:
def extract_country_code(text):
    """Safely extracts the two-letter country code from the address JSON."""
    if not text:
        return ""
    try:
        data = json.loads(text)
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
            return data[0].get('country', "")
        return ""
    except (json.JSONDecodeError, TypeError):
        return ""

def get_base_domain(text_list):
    """
    Extract and normalize websites

    Parameters
    ----------
    text_list : str
        List of websites as a string

    Returns
    -------
    set : A set holding the first website in the list
    """
    if not text_list:
        return ""

    try:
        data = json.loads(text_list)

        if isinstance(data, list) and len(data) > 0:
            url_string = None
            first_element = data[0]

            url_string = first_element

            if url_string and isinstance(url_string, str):
                # Use tldextract to get the domain and TLD (ignoring subdomains)
                extracted = tldextract.extract(url_string)

                # Check if we got a valid domain back
                if extracted.domain and extracted.suffix:
                    return f"{extracted.domain}.{extracted.suffix}"

        return ""
    except (json.JSONDecodeError, TypeError):
        return ""

def create_text_representation(row, column, key, base=False):
    """
    Create a clean, unified text description for a place record.

    Parameters
    ----------
    row : pd.Series
        One row from the dataset.
    base : bool
        If True, use the 'base_' columns (the reference place).
        If False, use the candidate columns.

    Returns
    -------
    str : A single concatenated string containing name, category, and address.
    """
    prefix = 'base_' if base else ''

    text = safe_json_parse(row[f'{prefix}{column}'], key=key)

    if not text:
      text = ""

    # Concatenate non-empty fields into one text string
    parts = [part for part in [text] if part]
    return ", ".join(parts)

def normalize_social_identifier(text_list):
    """
    Extract and normalize socials

    Parameters
    ----------
    text_list : str
        List of socials as a string

    Returns
    -------
    set : A set of all normalized socials
    """
    identifiers = set()
    if not text_list:
        return identifiers

    try:
        data = json.loads(text_list)

        if isinstance(data, list):
            for id_string in data:

                if id_string:
                    # Basic normalization: lowercase, strip whitespace, strip protocol/www
                    normalized_id = id_string.lower().strip().strip('/').replace('https://', '').replace('http://', '').lstrip('www.')
                    if normalized_id:
                        identifiers.add(normalized_id)

    except (json.JSONDecodeError, TypeError):
        pass

    return identifiers

from postal.parser import parse_address
from postal.expand import expand_address

def get_all_categories(text):
    """
    Safely parses the category JSON and returns a set containing the primary
    category and all alternate categories (lowercased and cleaned).
    """
    categories = set()
    if not text:
        return categories
    try:
        data = json.loads(text)

        if isinstance(data, dict):
            # 1. Add Primary Category
            primary = data.get('primary')
            if primary:
                categories.add(primary.lower().strip())

            # 2. Add Alternate Categories
            alternates = data.get('alternate')
            if isinstance(alternates, list):
                for alt in alternates:
                    if isinstance(alt, str):
                        categories.add(alt.lower().strip())

        return categories

    except (json.JSONDecodeError, TypeError):
        return categories

def normalize_social_identifier(text_list):
    """
    Extract and normalize socials

    Parameters
    ----------
    text_list : str
        List of socials as a string

    Returns
    -------
    set : A set of all normalized socials
    """
    identifiers = set()
    if not text_list:
        return identifiers

    try:
        data = json.loads(text_list)

        if isinstance(data, list):
            for id_string in data:

                if id_string:
                    # Basic normalization: lowercase, strip whitespace, strip protocol/www
                    normalized_id = id_string.lower().strip().strip('/').replace('https://', '').replace('http://', '').lstrip('www.')
                    if normalized_id:
                        identifiers.add(normalized_id)

    except (json.JSONDecodeError, TypeError):
        pass

    return identifiers

def get_base_domain(text_list):
    """
    Extract and normalize websites

    Parameters
    ----------
    text_list : str
        List of websites as a string

    Returns
    -------
    set : A set holding the first website in the list
    """
    if not text_list:
        return ""

    try:
        data = json.loads(text_list)

        if isinstance(data, list) and len(data) > 0:
            url_string = None
            first_element = data[0]

            url_string = first_element

            if url_string and isinstance(url_string, str):
                # Use tldextract to get the domain and TLD (ignoring subdomains)
                extracted = tldextract.extract(url_string)

                # Check if we got a valid domain back
                if extracted.domain and extracted.suffix:
                    return f"{extracted.domain}.{extracted.suffix}"

        return ""
    except (json.JSONDecodeError, TypeError):
        return ""

def extract_and_clean_number(address_text: str) -> str:
    """
    Safely extracts the primary numerical identifier (usually the street number)
    from a cleaned address string. Returns only digits and letters that precede
    the first main space, or the first number found.
    """
    if not isinstance(address_text, str) or not address_text:
        return ""

    # Simple strategy: find the first sequence of numbers/letters before the first space
    # Example: "12720 4th ave w" -> "12720"
    # Example: "Ground Floor, Kalyani Chambers, Plot No 80, Nigdi" -> "Ground" (Need a better parser)

    # A more robust strategy for US/Western addresses: find the initial number sequence.
    match = re.match(r'^[\d\s\w]+', address_text.lower().strip())
    if match:
        # Extract the whole starting segment (e.g., "12720 4th")
        starting_segment = match.group(0).split()

        # Take the first token that contains digits, which is usually the house number
        for token in starting_segment:
            if re.search(r'\d', token):
                # Clean the token: remove trailing letters often seen in addresses (e.g., 'A', 'Bldg C')
                # This ensures '123A' matches '123' if the letter is noise.
                return re.sub(r'[a-z]+$', '', token.strip('.,-'))

    return ""

Data Loading and Cleaning

In [11]:
from rapidfuzz import distance
from rapidfuzz import utils # Used for fast string normalization

file_path = "samples_3k_project_c_updated.parquet"
try:
    df = pd.read_parquet(file_path)
    print(f"Dataset loaded successfully. Shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found. Please upload it before proceeding.")
    exit()

# Apply the function to create two new columns
tqdm.pandas(desc="Creating text representations")
df['name'] = df.apply(lambda row: create_text_representation(row, "names", "primary", base=False), axis=1)
df['base_name'] = df.apply(lambda row: create_text_representation(row, "names", "primary", base=True), axis=1)

df['name_match'] = (
    df['name'].astype(str).str.lower() ==
    df['base_name'].astype(str).str.lower()
)

df['address'] = df.apply(lambda row: create_text_representation(row, "addresses", "freeform", base=False), axis=1)
df['base_address'] = df.apply(lambda row: create_text_representation(row, "addresses", "freeform", base=True), axis=1)

df['confidence_product'] = df['confidence'] * df['base_confidence']

def normalize_address(address_str):
    """
    Uses libpostal to generate a standardized list of address representations.
    We return the first (usually the most common) expanded form.
    """
    if pd.isna(address_str):
        return address_str

    # Generate expanded forms (e.g., 'st' instead of 'street', lowercase)
    expanded_forms = expand_address(address_str)

    # Return the first (most common) expanded form
    return expanded_forms[0] if expanded_forms else address_str

df['country_code'] = df.apply(lambda row: create_text_representation(row, "addresses", "country", base=False), axis=1)
df['base_country_code'] = df.apply(lambda row: create_text_representation(row, "addresses", "country", base=True), axis=1)

# Apply normalization to your existing address columns

df['address'] = df['address'].apply(normalize_address)
df['base_address'] = df['base_address'].apply(normalize_address)

df['category'] = df.apply(lambda row: create_text_representation(row, "categories", "primary", base=False), axis=1)
df['base_category'] = df.apply(lambda row: create_text_representation(row, "categories", "primary", base=True), axis=1)

df['locality'] = df.apply(lambda row: create_text_representation(row, "addresses", "locality", base=False), axis=1)
df['base_locality'] = df.apply(lambda row: create_text_representation(row, "addresses", "locality", base=True), axis=1)

df['country'] = df.apply(lambda row: create_text_representation(row, "addresses", "country", base=False), axis=1)
df['base_country'] = df.apply(lambda row: create_text_representation(row, "addresses", "country", base=True), axis=1)

df['postcode'] = df.apply(lambda row: create_text_representation(row, "addresses", "postcode", base=False), axis=1)
df['base_postcode'] = df.apply(lambda row: create_text_representation(row, "addresses", "postcode", base=True), axis=1)

df['region'] = df.apply(lambda row: create_text_representation(row, "addresses", "region", base=False), axis=1)
df['base_region'] = df.apply(lambda row: create_text_representation(row, "addresses", "region", base=True), axis=1)

# Compare normalized phone numbers

df['candidate_phone_norm'] = df.apply(
    lambda row: normalize_phone(row['phones'], row['country_code']),
    axis=1
)
df['base_phone_norm'] = df.apply(
    lambda row: normalize_phone(row['base_phones'], row['base_country_code']),
    axis=1
)

df['phone_match'] = (
    (df['candidate_phone_norm'] != "") &
    (df['base_phone_norm'] != "") &
    (df['candidate_phone_norm'] == df['base_phone_norm'])
).astype(int)

print('Total phone matches:', df['phone_match'].sum())

df['exact_phone_match_predict'] = (
    (df['phone_match'] == 1) &

    (df['label'] == 1)
)

print("Exact phone match correct predictions:", df['exact_phone_match_predict'].sum())

# Compare websites for binary match feature
df['website'] = df['websites'].apply(get_base_domain)
df['base_website'] = df['base_websites'].apply(get_base_domain)

df['website_match'] = (
    (df['base_website'] != "") &
    (df['website'] != "") &
    (df['website'] == df['base_website'])
).astype(int)

# Match any categories
df['category_set'] = df['categories'].apply(get_all_categories)
df['base_category_set'] = df['base_categories'].apply(get_all_categories)

df['category_match'] = df.apply(
    lambda row: 1 if row['category_set'].intersection(row['base_category_set']) else 0,
    axis=1
)

# Get the differences in length of the names of the two places
df['name_length_diff'] = np.abs(
    df['name'].str.len() - df['base_name'].str.len()
)

# Get the differences in length of the names of the two places
df['name_word_diff'] = np.abs(
    df['name'].apply(lambda x: len(x.split())) - df['base_name'].apply(lambda x: len(x.split()))
)

from rapidfuzz import fuzz

def calculate_fuzzy_features(name1: str, name2: str) -> dict:
    """
    Calculates key fuzzy ratios between two strings, normalized to 0.0 to 1.0.
    """
    # Ensure inputs are clean (lowercase and stripped of leading/trailing space)
    str1 = str(name1).lower().strip()
    str2 = str(name2).lower().strip()

    # Check if either string is empty to prevent errors or false high scores
    if not str1 or not str2:
        return {
            "ratio": 0.0,
            "partial_ratio": 0.0,
            "token_set_ratio": 0.0
        }

    return {
        # 1. Standard Similarity: Measures overall character similarity
        "ratio": fuzz.ratio(str1, str2) / 100.0,

        # 2. Partial Similarity: Finds the best matching substring (good for long vs. short)
        "partial_ratio": fuzz.partial_ratio(str1, str2) / 100.0,

        # 3. Token Set Ratio: Ignores word order and redundant/extra words (BEST for record linkage)
        "token_set_ratio": fuzz.token_set_ratio(str1, str2) / 100.0,
    }

# Apply the function to create a DataFrame of the scores
fuzzy_scores_df = df.apply(
    lambda row: pd.Series(
        calculate_fuzzy_features(row['name'], row['base_name'])
    ),
    axis=1
)

df = pd.concat([df, fuzzy_scores_df], axis=1)

# Apply the function to create a DataFrame of the scores
fuzzy_scores_addr_df = df.apply(
    lambda row: pd.Series(
        calculate_fuzzy_features(row['address'], row['base_address'])
    ),
    axis=1
)

fuzzy_scores_addr_df = fuzzy_scores_addr_df.rename(columns={
    'ratio': 'addr_ratio',
    'partial_ratio': 'addr_partial_ratio',
    'token_set_ratio': 'addr_token_set_ratio'
})

df = pd.concat([df, fuzzy_scores_addr_df], axis=1)

Dataset loaded successfully. Shape: (3000, 23)
Total phone matches: 1183
Exact phone match correct predictions: 1084


In [14]:
TARGET_COMPONENTS = ['house_number', 'road']
from typing import Dict, List, Tuple

def get_parsed_component_dict(address_string: str) -> Dict[str, str]:
    """
    Parses an address string and returns a dictionary containing only the
    house_number and road (street name), converted to uppercase.
    """
    if not address_string:
        return {}

    parsed_list: List[Tuple[str, str]] = parse_address(address_string)

    # Initialize dictionary with None for all target components
    components = {comp: None for comp in TARGET_COMPONENTS}

    for value, label in parsed_list:
        if label in TARGET_COMPONENTS:
            # We only keep the first instance of a component
            if components[label] is None:
                # Store cleaned, stripped, and uppercased value
                components[label] = str(value).strip().upper()

    return components

def create_road_match_features(df):
    """
    Applies libpostal parsing to 'address' and 'base_address' and creates
    two binary match features: 'match_house_num' and 'match_road'.
    """

    # 1. Apply parsing to both address columns to get structured components

    # Parse primary address ('address')
    parsed_a = df['address'].apply(get_parsed_component_dict)
    df['num'] = parsed_a.apply(lambda x: x.get('house_number'))
    df['road'] = parsed_a.apply(lambda x: x.get('road'))

    # Parse base address ('base_address')
    parsed_b = df['base_address'].apply(get_parsed_component_dict)
    df['base_num'] = parsed_b.apply(lambda x: x.get('house_number'))
    df['base_road'] = parsed_b.apply(lambda x: x.get('road'))

    # 2. Create the two match features (0 or 1)

    # Feature 1: Exact House Number Match
    # We must first ensure all values are strings and clean up NaNs/None for reliable comparison
    num_a = df['num'].astype(str).str.strip().fillna('')
    num_b = df['base_num'].astype(str).str.strip().fillna('')

    # 'match_house_num' is 1 only if both extracted numbers are non-empty and identical
    df['match_num'] = ((num_a != '') & (num_a == num_b)).astype(int)
    df['mismatch_num'] = ((num_a != '') & (num_b != '') & (num_a != num_b)).astype(int)

    # Feature 2: Exact Road/Street Name Match
    road_a = df['road'].astype(str).str.strip().fillna('')
    road_b = df['base_road'].astype(str).str.strip().fillna('')

    # 'match_road' is 1 only if both extracted road names are non-empty and identical
    df['match_road'] = ((road_a != '') & (road_a == road_b)).astype(int)

    return df

df = create_road_match_features(df)

In [15]:
# ===================================================================
# Step 3: Create Proxy Labels & Split Data (Avoiding Leakage)
# ===================================================================

# print(f"\nLabel distribution:\n{df['label'].value_counts()}")
# print(f"Positive sample (match) ratio: {df['label'].mean():.2%}")

X = df[['name', 'base_name', 'address', 'base_address', 'phone_match',
        'match_num', 'match_road', 'category', 'base_category',
        'locality', 'base_locality', 'region', 'base_region', 'confidence_product',
        'addr_partial_ratio', 'addr_token_set_ratio', 'partial_ratio', 'token_set_ratio',
        'name_length_diff', 'website_match', 'category_match', 'mismatch_num',
        'country', 'base_country', 'name_word_diff'
        ]]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

Login if using restricted model

In [16]:
from huggingface_hub import login
login()

In [23]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score, precision_recall_curve, auc, accuracy_score, classification_report
import time
import gc

# Processes features in the dataframe for training and testing
def process_feature(X_set: pd.DataFrame, set_name: str, embedder: SentenceTransformer, column: str) -> Tuple[Dict, List[np.ndarray], np.ndarray]:

    text_A = X_set[column].apply(lambda x: x.strip() if x else '')
    text_B = X_set["base_" + column].apply(lambda x: x.strip() if x else '')

    empty_mask = (text_A == '') & (text_B == '')

    instruction = ''
    emb = None
    base_emb = None
    # Different embedding format using multiple columns for address column
    if column == 'address':
      emb = embedder.encode(
          (text_A + X_set['locality'] + X_set['region'] + X_set['country']).tolist(),
          convert_to_tensor=True,
          show_progress_bar=False,
          device=embedder.device
      )
      base_emb = embedder.encode(
          (text_B + X_set['base_locality'] + X_set['base_region'] + X_set['base_country']).tolist(),
          convert_to_tensor=True,
          show_progress_bar=False,
          device=embedder.device
      )
    else:
      emb = embedder.encode(
          (instruction + text_A).tolist(),
          convert_to_tensor=True,
          show_progress_bar=False,
          device=embedder.device
      )
      base_emb = embedder.encode(
          (instruction + text_B).tolist(),
          convert_to_tensor=True,
          show_progress_bar=False,
          device=embedder.device
      )
    sim_scores = cos_sim(emb, base_emb).diag().cpu().numpy()

    return [sim_scores]

def reshape_similarities(sims):
  reshaped_sims = []
  for sim in sims:
      reshaped_sims.append(sim.reshape(-1, 1))
  return reshaped_sims

def reshape_and_stack_features(X_data, sim_list, discrete_cols):
    """Reshapes and stacks all semantic and discrete features from the full dataset."""
    discrete_features_list = [X_data[col].values.reshape(-1, 1) for col in discrete_cols]

    return np.hstack(discrete_features_list + sim_list)

classifier = GradientBoostingClassifier(
    n_estimators=496, learning_rate=0.01, max_depth=6, subsample=0.7,
    max_features=0.7, min_samples_leaf=7, random_state=42
)

Test a wide array of models and their performance

In [25]:
small_models = [
    'intfloat/multilingual-e5-base',
    'BAAI/bge-m3',
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    'Alibaba-NLP/gte-multilingual-base',
    'google/embeddinggemma-300m',
    'Qwen/Qwen3-Embedding-0.6B',
    'intfloat/multilingual-e5-large'
]
feature_list = ['name', 'category', 'address']
DISCRETE_COLUMNS = [
    'phone_match', 'confidence_product', 'category_match', 'name_length_diff',
    'website_match', 'token_set_ratio', 'partial_ratio',
    'addr_partial_ratio', 'addr_token_set_ratio', 'name_word_diff'
]

# List to store all model results
results_list = []

for model_name in small_models:
    print(f"\n--- Testing Model: {model_name} ---")

    try:
        try:
            embedder = SentenceTransformer(model_name, device='cuda' if torch.cuda.is_available() else 'cpu', trust_remote_code=True)
        except NameError:
            embedder = SentenceTransformer(model_name, trust_remote_code=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model {model_name}: {e}. Skipping.")
        continue

    # Generate Semantic Embeddings and Similarities ---
    train_similarity_list = []
    test_similarity_list = []
    total_time = 0
    for feature in feature_list:
        train_similarity_list = train_similarity_list + process_feature(X_train, 'train', embedder, feature)

        # Start total test set embedding time
        embed_time_start = time.perf_counter()
        test_similarity_list = test_similarity_list + process_feature(X_test, 'test', embedder, feature)

        # Add total test set embedding time to counter
        total_time += time.perf_counter() - embed_time_start

    reshaped_train_sim = reshape_similarities(train_similarity_list)
    reshaped_test_sim = reshape_similarities(test_similarity_list)

    # Stack Features ---
    X_train_final = reshape_and_stack_features(X_train, reshaped_train_sim, DISCRETE_COLUMNS)
    X_test_final = reshape_and_stack_features(X_test, reshaped_test_sim, DISCRETE_COLUMNS)

    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_f1_scores = cross_val_score(
        classifier, X_train_final, y_train, cv=cv_strategy, scoring='f1', n_jobs=-1
    )

    # Train and Evaluate Classifier ---
    print("Training Gradient Boosting Classifier...")
    classifier.fit(X_train_final, y_train)


    eval_time_start = time.perf_counter()
    print("Evaluating on test set...")
    y_pred_proba = classifier.predict_proba(X_test_final)[:, 1]
    y_pred = classifier.predict(X_test_final)

    # Add total test evaluation time to counter
    total_time += time.perf_counter() - eval_time_start

    # Calculate Metrics
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall, precision)
    average_f1 = cv_f1_scores.mean()

    # Display Scores
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Average F1 CV Score: {average_f1:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  PR-AUC:   {pr_auc:.4f}")
    print(f"  Total Test Set Processing Time: {total_time:.2f}s")

    # --- 5. Save Results ---
    results_list.append({
        'Model Name': model_name,
        'F1-Score': f1,
        'Average F1 CV Scores': average_f1,
        'Accuracy': accuracy,
        'PR-AUC': pr_auc,
        'Total Test Time': total_time
      }),

    del embedder
    gc.collect()
    torch.cuda.empty_cache()

# --- 6. Create and Save CSV ---
print("\n" + "="*80)
print("All Evaluations Complete. Generating Summary CSV.")
print("="*80)
results_df = pd.DataFrame(results_list)

# Display the final summary table
print("\n--- Summary of Model Performance ---")
print(results_df.sort_values(by='F1-Score', ascending=False).to_markdown(index=False, floatfmt=".4f"))

# Save the DataFrame to a CSV file
output_csv_path = 'small_embedding_model_comparison_results.csv'
results_df.to_csv(output_csv_path, index=False)
print(f"\nResults saved successfully to: **{output_csv_path}**")


--- Testing Model: intfloat/multilingual-e5-base ---
Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.8991
  Average F1 CV Score: 0.8911
  Accuracy: 0.8750
  PR-AUC:   0.9677
  Total Test Set Processing Time: 3.18s

--- Testing Model: BAAI/bge-m3 ---
Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.9015
  Average F1 CV Score: 0.8974
  Accuracy: 0.8783
  PR-AUC:   0.9691
  Total Test Set Processing Time: 11.45s

--- Testing Model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 ---
Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.9024
  Average F1 CV Score: 0.8935
  Accuracy: 0.8800
  PR-AUC:   0.9667
  Total Test Set Processing Time: 1.25s

--- Testing Model: Alibaba-NLP/gte-multilingual-base ---


Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.9015
  Average F1 CV Score: 0.8945
  Accuracy: 0.8783
  PR-AUC:   0.9691
  Total Test Set Processing Time: 4.53s

--- Testing Model: google/embeddinggemma-300m ---
Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.8993
  Average F1 CV Score: 0.8982
  Accuracy: 0.8783
  PR-AUC:   0.9706
  Total Test Set Processing Time: 6.24s

--- Testing Model: Qwen/Qwen3-Embedding-0.6B ---
Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.8946
  Average F1 CV Score: 0.8954
  Accuracy: 0.8700
  PR-AUC:   0.9688
  Total Test Set Processing Time: 19.70s

--- Testing Model: intfloat/multilingual-e5-large ---
Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.8937
  Average F1 CV Score: 0.9002
  Accuracy: 0.8683
  PR-AUC:   0.9652

In [ ]:
large_models = ['nvidia/llama-embed-nemotron-8b', 'Qwen/Qwen3-Embedding-4B', 'Qwen/Qwen3-Embedding-8B']
feature_list = ['name', 'category', 'address']
DISCRETE_COLUMNS = [
    'phone_match', 'confidence_product', 'category_match', 'name_length_diff',
    'website_match', 'token_set_ratio', 'partial_ratio', 'mismatch_num',
    'addr_partial_ratio', 'addr_token_set_ratio'
]
# List to store all model results
results_list = []

for model_name in large_models:
    print(f"\n--- Testing Model: {model_name} ---")

    try:
        try:
            embedder = SentenceTransformer(model_name, device='cuda' if torch.cuda.is_available() else 'cpu', trust_remote_code=True)
        except NameError:
            embedder = SentenceTransformer(model_name, trust_remote_code=True)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model {model_name}: {e}. Skipping.")
        continue

    # Generate Semantic Embeddings and Similarities ---
    train_similarity_list = []
    test_similarity_list = []
    total_time = 0
    for feature in feature_list:
        train_similarity_list = train_similarity_list + process_feature(X_train, 'train', embedder, feature)

        # Start total test set embedding time
        embed_time_start = time.perf_counter()
        test_similarity_list = test_similarity_list + process_feature(X_test, 'test', embedder, feature)

        # Add total test set embedding time to counter
        total_time += time.perf_counter() - embed_time_start

    reshaped_train_sim = reshape_similarities(train_similarity_list)
    reshaped_test_sim = reshape_similarities(test_similarity_list)

    # Stack Features ---
    X_train_final = reshape_and_stack_features(X_train, reshaped_train_sim, DISCRETE_COLUMNS)
    X_test_final = reshape_and_stack_features(X_test, reshaped_test_sim, DISCRETE_COLUMNS)

    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_f1_scores = cross_val_score(
        classifier, X_train_final, y_train, cv=cv_strategy, scoring='f1', n_jobs=-1
    )

    # Train and Evaluate Classifier ---
    print("Training Gradient Boosting Classifier...")
    classifier.fit(X_train_final, y_train)


    eval_time_start = time.perf_counter()
    print("Evaluating on test set...")
    y_pred_proba = classifier.predict_proba(X_test_final)[:, 1]
    y_pred = classifier.predict(X_test_final)

    # Add total test evaluation time to counter
    total_time += time.perf_counter() - eval_time_start

    # Calculate Metrics
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc = auc(recall, precision)
    average_f1 = cv_f1_scores.mean()

    # Display Scores
    print(f"  F1-Score: {f1:.4f}")
    print(f"  Average F1 CV Score: {average_f1:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  PR-AUC:   {pr_auc:.4f}")
    print(f"  Total Test Set Processing Time: {total_time:.2f}s")

    # --- 5. Save Results ---
    results_list.append({
        'Model Name': model_name,
        'F1-Score': f1,
        'Average F1 CV Scores': average_f1,
        'Accuracy': accuracy,
        'PR-AUC': pr_auc,
        'Total Test Time': total_time
      }),

    del embedder
    gc.collect()
    torch.cuda.empty_cache()

# --- 6. Create and Save CSV ---
print("\n" + "="*80)
print("All Evaluations Complete. Generating Summary CSV.")
print("="*80)
results_df = pd.DataFrame(results_list)

# Display the final summary table
print("\n--- Summary of Model Performance ---")
print(results_df.sort_values(by='F1-Score', ascending=False).to_markdown(index=False, floatfmt=".4f"))

# Save the DataFrame to a CSV file
output_csv_path = 'large_embedding_model_comparison_results.csv'
results_df.to_csv(output_csv_path, index=False)
print(f"\nResults saved successfully to: **{output_csv_path}**")


--- Testing Model: nvidia/llama-embed-nemotron-8b ---


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully.
Training Gradient Boosting Classifier...
Evaluating on test set...
  F1-Score: 0.8300
  Average F1 CV Score: 0.8448
  Accuracy: 0.8000
  PR-AUC:   0.9348
  Total Test Set Processing Time: 46.22s

--- Testing Model: Qwen/Qwen3-Embedding-4B ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully.


KeyboardInterrupt: 